In [ ]:
from modelzipper.tutils import *
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
import random
from multiprocessing import Pool, cpu_count

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
dir_path = "/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/llama"
dataset_path = "/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/filter_en"
# tokenizer = AutoTokenizer.from_pretrained("/data/hf_models/Meta-Llama-3.1-8B-Instruct")
# dir_path = "/data/pub_data/check_inference/check_inference/llama"
# dataset_path = "/data/pub_data/processed_multi_hop/filter_en"
file_names = auto_read_dir(dir_path)
file_names.sort()
content_drop_1 = auto_read_data(os.path.join(dir_path, file_names[0]))
content_drop_2 = auto_read_data(os.path.join(dir_path, file_names[1]))
content_drop_3 = auto_read_data(os.path.join(dir_path, file_names[2]))

all_file_names = auto_read_dir(dataset_path)
content = []
for file_name in all_file_names:
    content.extend(auto_read_data(os.path.join(dataset_path, file_name)))

### 下面是补充不完整的prompt

In [ ]:
def process_data_item(args):
    item, tokenizer, drop_num = args
    prompt, full_evi_prompt, answer, selected_ids = process_item(item, drop_num)

    input_data = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        add_generation_prompt=True, tokenize=False,
    )

    meta_data = copy.deepcopy(item)
    meta_data.pop('concat_content')
    meta_data.pop('instruction_format')
    meta_data.pop('answer')
    return {
        "prompt": prompt,
        "ori_prompt": full_evi_prompt,
        "message": input_data,
        "answer": answer,
        "meta_data": meta_data, 
        "selected_ids": selected_ids,
    }


def process_data(content, tokenizer, drop_num=1, num_workers=24):
    if num_workers is None:
        num_workers = max(1, cpu_count() - 1)  # Default: use all CPUs except one

    # Prepare arguments for process_data_item
    args = [(item, tokenizer, drop_num) for item in content]

    # Use multiprocessing to parallelize processing
    with Pool(num_workers) as pool:
        all_inference_content = list(tqdm(pool.imap(process_data_item, args), total=len(content)))

    return all_inference_content

### 下面的数据集仅用来进行测试开发使用，一共32条训练，32条测试

In [ ]:
# 首先从每个数据集中取相同数目的数据出来
all_training_data = []

all_content_data = content_drop_1 + content_drop_2 + content_drop_3
all_content_data = all_content_data[:512]
for item in all_content_data:
    all_training_data.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["pred"][0]}
        ],
    })

dataset = Dataset.from_list(all_training_data)

# 从中随机抽取 32 条作为 validation 数据
validation_size = 32

# 打乱数据集索引并取前 32 条作为 validation
indices = list(range(len(dataset)))
random.shuffle(indices)

validation_indices = indices[:validation_size]
train_indices = indices[validation_size:]

# 使用 Hugging Face 的 select 方法创建新的训练集和验证集
train_dataset = dataset.select(train_indices)
validation_dataset = dataset.select(validation_indices)

# 将 train 和 validation 数据集保存到一个 dict 中
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

# 打印数据集长度验证
print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Validation dataset size: {len(dataset_dict['validation'])}")

# 保存到本地
dataset_dict.save_to_disk("/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/train_data/merge_v1_dev")

In [ ]:
### 构造训练数据集

In [ ]:
content_drop_1[1]['answer'] == content_drop_2[1]['answer'] == content_drop_3[1]['answer']

In [ ]:
# 首先从每个数据集中取相同数目的数据出来
post_proc_drop_1, post_proc_drop_2, post_proc_drop_3 = [], [], []

# 从每个子集里面sample 3200条数据再合并到一起
random.shuffle(content_drop_1)
random.shuffle(content_drop_2)
random.shuffle(content_drop_3)


cnt = 0
for item in tqdm(content_drop_1): ## 首先检查输出是否符合预期格式
    model_pred = item["pred"][0].split('\n\n')[0]
    if len(model_pred) < 100:
        continue
    post_proc_drop_1.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": model_pred}
        ],
    })
    cnt += 1
    if cnt >= 3200:
        break
print(f"Post_proc_drop_1 size: {len(post_proc_drop_1)}")

cnt = 0
for item in tqdm(content_drop_2): ## 首先检查输出是否符合预期格式
    model_pred = item["pred"][0].split('\n\n')[0]
    if len(model_pred) < 100:
        continue
    post_proc_drop_2.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": model_pred}
        ],
    })
    cnt += 1
    if cnt >= 3200:
        break
print(f"Post_proc_drop_2 size: {len(post_proc_drop_2)}")

cnt = 0
for item in tqdm(content_drop_3): ## 首先检查输出是否符合预期格式
    model_pred = item["pred"][0].split('\n\n')[0]
    if len(model_pred) < 100:
        continue
    post_proc_drop_3.append({
        "prompt": item["prompt"],
        "chosen": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": item["answer"]}
        ],
        "rejected": [
            {"role": "user", "content": item["prompt"]}, 
            {"role": "assistant", "content": model_pred}
        ],
    })
    cnt += 1
    if cnt >= 3200:
        break
print(f"post_proc_drop_3 size: {len(post_proc_drop_3)}")


all_training_data = post_proc_drop_1 + post_proc_drop_2 + post_proc_drop_3
random.shuffle(all_training_data)


# 确保所有字段格式一致，处理潜在的问题
for record in all_training_data:
    if not isinstance(record["chosen"], list):
        record["chosen"] = [record["chosen"]]
    if not isinstance(record["rejected"], list):
        record["rejected"] = [record["rejected"]]

dataset = Dataset.from_list(all_training_data)

# 从中随机抽取 600 条作为 validation 数据
validation_size = 600

# 打乱数据集索引并取前 400 条作为 validation
indices = list(range(len(dataset)))
random.shuffle(indices)

validation_indices = indices[:validation_size]
train_indices = indices[validation_size:]

# 使用 Hugging Face 的 select 方法创建新的训练集和验证集
train_dataset = dataset.select(train_indices)
validation_dataset = dataset.select(validation_indices)

# 将 train 和 validation 数据集保存到一个 dict 中
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

# 打印数据集长度验证
print(f"Train dataset size: {len(dataset_dict['train'])}")
print(f"Validation dataset size: {len(dataset_dict['validation'])}")

# 保存到本地
dataset_dict.save_to_disk("/mnt/petrelfs/tangzecheng/local_data/processed_multi_hop/random_drop/train_qwen_data/merge_v1")